In [46]:
import yfinance as yf
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from patsy import dmatrix

In [61]:
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    return stock

In [62]:
data = yf.download("^DJI 000001.SS ^GSPC", start = "2020-01-22")
data.dropna()

[*********************100%***********************]  3 of 3 completed


Adj Close                                   Close                \
              000001.SS          ^DJI        ^GSPC    000001.SS          ^DJI   
Date                                                                            
2020-01-22  3060.754883  29186.269531  3321.750000  3060.754883  29186.269531   
2020-01-23  2976.528076  29160.089844  3325.540039  2976.528076  29160.089844   
2020-02-03  2746.605957  28399.810547  3248.919922  2746.605957  28399.810547   
2020-02-04  2783.288086  28807.630859  3297.590088  2783.288086  28807.630859   
2020-02-05  2818.087891  29290.849609  3334.689941  2818.087891  29290.849609   
2020-02-06  2866.510010  29379.769531  3345.780029  2866.510010  29379.769531   
2020-02-07  2875.959961  29102.509766  3327.709961  2875.959961  29102.509766   
2020-02-10  2890.488037  29276.820312  3352.090088  2890.488037  29276.820312   
2020-02-11  2901.669922  29276.339844  3357.750000  2901.669922  29276.339844   
2020-02-12  2926.898926  29551.419922  3379.449951  2926.898926  29551.419922   
2020-02-13  2906.073975  29423.310547  3373.939941  2906.073975  29423.310547   
2020-02-14  2917.008057  29398.080078  3380.159912  2917.008057  29398.080078   
2020-02-18  2984.971924  29232.189453  3370.290039  2984.971924  29232.189453   
2020-02-19  2975.402100  29348.029297  3386.149902  2975.402100  29348.029297   
2020-02-20  3030.154053  29219.980469  3373.229980  3030.154053  29219.980469   
2020-02-21  3039.668945  28992.410156  3337.750000  3039.668945  28992.410156   
2020-02-24  3031.232910  27960.800781  3225.889893  3031.232910  27960.800781   
2020-02-25  3013.050049  27081.359375  3128.209961  3013.050049  27081.359375   
2020-02-26  2987.928955  26957.589844  3116.389893  2987.928955  26957.589844   
2020-02-27  2991.330078  25766.640625  2978.760010  2991.330078  25766.640625   
2020-02-28  2880.303955  25409.359375  2954.219971  2880.303955  25409.359375   
2020-03-02  2970.930908  26703.320312  3090.229980  2970.930908  26703.320312   
2020-03-03  2992.896973  25917.410156  3003.370117  2992.896973  25917.410156   
2020-03-04  3011.666016  27090.859375  3130.120117  3011.666016  27090.859375   
2020-03-05  3071.677002  26121.279297  3023.939941  3071.677002  26121.279297   
2020-03-06  3034.510986  25864.779297  2972.370117  3034.510986  25864.779297   
2020-03-09  2943.291016  23851.019531  2746.560059  2943.291016  23851.019531   
2020-03-10  2996.761963  25018.160156  2882.229980  2996.761963  25018.160156   
2020-03-11  2968.517090  23553.220703  2741.379883  2968.517090  23553.220703   
2020-03-12  2923.486084  21200.619141  2480.639893  2923.486084  21200.619141   
2020-03-13  2887.427002  23185.619141  2711.020020  2887.427002  23185.619141   
2020-03-16  2789.253906  20188.519531  2386.129883  2789.253906  20188.519531   
2020-03-17  2779.641113  21237.380859  2529.189941  2779.641113  21237.380859   
2020-03-18  2728.756104  19898.919922  2398.100098  2728.756104  19898.919922   
2020-03-19  2702.129883  20087.189453  2409.389893  2702.129883  20087.189453   
2020-03-20  2745.617920  19173.980469  2304.919922  2745.617920  19173.980469   

                                High                                     Low  \
                  ^GSPC    000001.SS          ^DJI        ^GSPC    000001.SS   
Date                                                                           
2020-01-22  3321.750000  3069.252930  29320.199219  3337.770020  3006.268066   
2020-01-23  3325.540039  3045.041016  29190.470703  3326.879883  2955.345947   
2020-02-03  3248.919922  2766.576904  28630.390625  3268.439941  2716.697998   
2020-02-04  3297.590088  2786.164062  28904.880859  3306.919922  2685.269043   
2020-02-05  3334.689941  2842.742920  29308.890625  3337.580078  2778.864014   
2020-02-06  3345.780029  2876.590088  29408.050781  3347.959961  2807.610107   
2020-02-07  3327.709961  2875.959961  29286.919922  3341.419922  2838.770020   
2020-02-10  3352.090088  2891.847900  29278.070312  33

In [63]:
data["China SSE"] = (data["High", "000001.SS"] + data["Low", "000001.SS"]) / 2
data["US DJI"] = (data["High", "^DJI"] + data["Low", "^DJI"]) / 2
data["US S&P 500"] = (data["High", "^GSPC"] + data["Low", "^GSPC"]) / 2
data = data.drop(columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'])
data = data.dropna().reset_index()
data

,Date,China SSE,US DJI,US S&P 500
,,,,
0,2020-01-22,3037.760498,29246.229492,3328.905029
1,2020-01-23,3000.193481,29078.725586,3314.375000
2,2020-02-03,2741.637451,28475.020508,3252.049927
3,2020-02-04,2735.716553,28800.810547,3293.765015
4,2020-02-05,2810.803467,29154.870117,3325.665039
5,2020-02-06,2842.100098,29327.490234,3341.174927
6,2020-02-07,2857.364990,29171.950195,3331.770020
7,2020-02-10,2871.448975,29136.865234,3335.015015
8,2020-02-11,2898.030029,29312.930664,3364.174927


In [84]:
COVID_DATASET = r"C:\Users\William\Documents\Github\Coronadex\Data\full_corona.csv"
cov_data = pd.read_csv(COVID_DATASET)
cov_data = cov_data.drop(columns={'new_cases', 'new_deaths', 'total_deaths'})
# cov_data = cov_data.sort_values('location')
cov_data["Current Total"] = cov_data.drop(columns = ["location"]).sum(axis=1)
cov_data = cov_data[["Current Total", "location"]].sort_values("location")
# full_data =
cov_data

,Current Total,location
0,0,Afghanistan
51,0,Afghanistan
50,0,Afghanistan
49,0,Afghanistan
48,0,Afghanistan
...,...,...
6038,7823,World
6092,2,Zambia
6090,2,Zambia
6091,2,Zambia
